# Experiment setup
Below we create a file that determines which simulations should be ran, for which graphs, and how often. It exports a .sh file that can be executed in the terminal. To run the code and the bash file, ensure that all the folders to store data exist locally. 

In [8]:
from pathlib import Path
import re

import numpy as np
import pandas as pd

summary_folder = Path("data/public_release/varying_symptom_rate")


records = [{"app_rate": np.round(app_rate, 6),
            "beta": np.round(beta, 2),
            "graph": graph,
            "method": method,
            "symptom_prob": np.round(symptom_prob,2),
            "runs_to_go": 5}                                     
            for beta in [0.2]
            for symptom_prob in [0.02]
            for app_rate in [0.2, 0.4, 0.6, 0.8]
            for method in [
                "recommender", 
                           "recommend_friends_with_p=0.5",
                           "degree",
                           "random"
            ]
            for graph in [
                "girgC_tau=2.3_alpha=1.3_deg=13.tar",
#                 "girgC_tau=2.3_alpha=2.3_deg=13.tar",
#                 "girgC_tau=3.3_alpha=1.3_deg=13.tar",
#                 "girgC_tau=3.3_alpha=2.3_deg=13.tar",
#                 "cm_tau=2.3_deg=13.tar",
#                 "cm_tau=3.3_deg=13.tar"
            ]]
records += [{"app_rate": np.round(app_rate, 6),
            "beta": np.round(beta, 2),
            "graph": graph,
            "method": method,
            "symptom_prob": np.round(symptom_prob,2),
            "runs_to_go": 5}                                     
            for beta in [0.2]
            for symptom_prob in [0.02]
            for app_rate in [0, 1]
            for method in [
#                 "recommender", 
#                 "recommend_friends_with_p=0.5",
#                 "degree",
                  "random"
            ]
            for graph in [
                "girgC_tau=2.3_alpha=1.3_deg=13.tar",
                "girgC_tau=2.3_alpha=2.3_deg=13.tar",
#               "girgC_tau=3.3_alpha=1.3_deg=13.tar",
#               "girgC_tau=3.3_alpha=2.3_deg=13.tar",
#               "cm_tau=2.3_deg=13.tar",
#               "cm_tau=3.3_deg=13.tar"
            ]]

run_df = pd.DataFrame.from_records(records)
coverages_df = pd.concat([pd.read_csv("optimized_coverages_cm.csv"), pd.read_csv("optimized_coverages_girgC.csv")])
run_df["target_coverage"] = np.round(run_df["app_rate"], 2)
coverages_df["graph"] = coverages_df["graph"].str[:-3]
coverages_df["target_coverage"] = np.round(coverages_df["target_coverage"], 2)
index_cols = ["graph", "method", "target_coverage"]
run_df = run_df.set_index(index_cols).join(coverages_df.set_index(index_cols), rsuffix='_initial').reset_index()
run_df["coverage"] = run_df["coverage"].fillna(run_df["app_rate"])
run_df["app_rate_initial"] = np.round(run_df["app_rate_initial"].fillna(run_df["app_rate"]), 6)
run_df["app_rate_initial_rounded"] = np.round(run_df["app_rate_initial"], 3)

    
def blow_up(record):    
    runs_to_go = record.pop("runs_to_go")    
    return [{**record, "run_id": i}  for i in range(1, runs_to_go + 1)]
run_df = pd.DataFrame.from_records([new_record 
                                   for old_record in run_df[run_df["runs_to_go"]>0].to_dict(orient="record") 
                                   for new_record in blow_up(old_record)])


records_todo = run_df.to_dict(orient="record")

record_to_line = lambda record: \
f"sem -j+0 \"python varying_app_rate.py \
data/graphs-christopher/{record['graph']}.gz \
{record['beta']} \
{record['symptom_prob']} \
{np.round(record['app_rate_initial'], 6)} \
{record['method']} \
data/public_release/varying_symptom_rate/{record['graph']}/{record['method']}/run={record['run_id']}_beta={record['beta']}_app_rate={np.round(record['app_rate_initial'],6)}_symptom_prob={record['symptom_prob']}#$(date +%s).tar.gz\"" 


    
lines = (["#!/bin/sh", "set -e", "\n"] + 
         [record_to_line(record) 
          for record in run_df.to_dict(orient="record")]
         + ["\n", "sem --wait"]
         )

with open("app-test.sh", "w") as f:
    f.writelines("\n".join(lines))

# Combining data
Below we read the generated data by the bash script and combine the results into a single large csv that can be read by the "Visualisations paper" notebook

In [10]:
paths = list(Path("data/public_release/varying_symptom_rate").glob("g*/*/s*.tar.gz"))

def read_path(summary_path):
    try:
        df = SpreadingStateHistory.load_bundled_summaries_to_time_df(summary_path,
                                                                     bundle_id=summary_path.name,
                                                                     description_params=["method",
                                                                                         "symptom_prob",
                                                                                         "infection_prob",
                                                                                         "average_degree",
                                                                                         "app_coverages",
                                                                                         "app_rate"])
        df["graph"] = summary_path.parent.parent.name
        df["Run_ID"] = df["Run_ID"] + "__" + df["Bundle_ID"] 
        df["app_method"] = df["method"]
    except:
        raise
        df = pd.DataFrame()
    return df

new_dfs = [read_path(summary_path) for summary_path in paths]


In [11]:
df = pd.concat(new_dfs)

# Enrich data
We enrich the data by rounding the coverage off the app, adding extra columns with different names for the strategy, computing hospital load, etc. 

In [14]:
def enrich_full_data(df):
    # In case of no/full app usage all systems coincide and results can be reused across scenarios
    mask = df["app_rate"].isin([0, 1]) & (df["app_method"]!="random")
    df = df[~mask]
    trivial_df = df[df["app_rate"].isin([0,1]) & (df["app_method"]=="random")].copy()
    for method in set(df["app_method"].unique()) - {"random"}:
        trivial_df["app_method"] = method
        trivial_df["method"] = method
        df = pd.concat([df, trivial_df])
    df = df.drop_duplicates()

    # the precise app_coverage for recommender systems are determined by initial app users that differ per run
    # to combine different runs with 'almost' same app_coverage two columns are added
    df.drop(columns=[c for c in df.columns if c.startswith("app_coverages_")], inplace=True)
    group_cols = ["app_method", "graph", "app_rate", "symptom_prob", "infection_prob"]
    df = df.set_index(group_cols).join(df.groupby(group_cols)[["app_coverages"]].mean(), rsuffix="_mean").reset_index()
    mask = ((df["app_coverages_mean"] % 0.05) > 0.025) 
    df["app_coverages_rounded"] = df["app_coverages_mean"] - (df["app_coverages_mean"] % 0.05) 
    df.loc[mask, "app_coverages_rounded"] = df["app_coverages_rounded"] + 0.05  
    df["app_coverages_rounded"] = np.round(df["app_coverages_rounded"], 2)
    # scale by 100 to display percentages instead of fractions
    df["app_coverages_mean_perc"] = 100*df["app_coverages_mean"]


    ########
    df["Quarantined"] = (df["Q_susceptible"] 
                         + df["Q_exposed"] 
                         + df["Q_infected"] 
                         + df["Symptomatic"] 
                         + df["Q_removed"])
    df["Newly Quarantined"] = (df["Newly Q_susceptible"] 
                                 + df["Newly Q_exposed"] 
                                 + df["Newly Q_infected"] 
                                 + df["Newly Symptomatic"] 
                                 + df["Newly Q_removed"])

    df["Strategy"] = (df["app_method"]
                      .replace("recommender", "Basic Recommender")
                      .replace("recommend_friends_with_p=0.5", "Ring Recommender")
                      .replace("random", "Random")
                      .replace("degree", "Degree"))
    df["Hospital"] = 0.05 * df["Symptomatic"] / df["symptom_prob"]
    df = df.sort_values(by=["graph",
                            "app_method", 
                            "infection_prob", 
                            "app_coverages_mean", 
                            "symptom_prob",
                            "Run_ID",
                            "Time"])
    return df


In [21]:
enriched_df = enrich_full_data(df)

# Compute KPIs
From this data we can extract the KPIs as presented in the paper

In [19]:
def create_kpi_df(df, upper_percentile=90, lower_percentile=10):
    df = df.copy()
    index_cols_setting = ["app_rate", 
                          "app_coverages_mean",
                          "app_coverages_rounded",
                          "app_coverages_mean_perc",
                          "Strategy",
                          "graph", 
                          "infection_prob", 
                          "symptom_prob"]
    index_cols_run = index_cols_setting + ["Bundle_ID", "Run_ID"]
    if upper_percentile < 100:
        upper = lambda x: np.percentile(x, upper_percentile)
    else:
        upper = np.max
    if lower_percentile > 0:
        lower = lambda x: np.percentile(x, lower_percentile)
    else:
        lower = np.min
        
    agg_functions = ["median", 
                     ('upper', upper),
                     ('lower', lower)]

    def percentage_in_quarantine(df):
        a = df.groupby(index_cols_run).agg({"Quarantined":"sum", "Time":"max"})
        a["Quar./person/year"] = 365 * a["Quarantined"] / (a["Time"] * 500000)
        a["Quar./person"] =  a["Quarantined"] / (500000)
        return a.reset_index().groupby(index_cols_setting)[["Quar./person/year", "Quar./person"]].agg(agg_functions)


    
    kpi_df = (df.groupby(index_cols_run).agg({"Removed":"max",
                                              "Symptomatic": "max",
                                              "Quarantined":"max",
                                              "Time": "max",
                                              "Hospital": "max"})
                        .reset_index()
                        .groupby(index_cols_setting)
                        .agg(agg_functions)
                        .join(percentage_in_quarantine(df))
                        .join(df.groupby(index_cols_run)[["Symptomatic"]]
                                .max()
                                .reset_index()
                                .set_index(index_cols_run + ["Symptomatic"])
                                .join(df[index_cols_run + ["Symptomatic", "Time"]].set_index(index_cols_run + ["Symptomatic"]), rsuffix="_r")
                                .reset_index()
                                .groupby(index_cols_setting)[["Time"]]
                                .agg(agg_functions)
                                .rename(columns={"Time": "Time of peak"}))
                        .rename(columns={"Symptomatic": "Symptomatic Max.", 
                                         "Quarantined": "Quarantined Max.",
                                         "Hospital": "Hospital Max.",
                                         "Removed": "Volume",
                                         "Time": "Time to epidemic end"})
                        .reset_index())    
    kpi_df["Time to epidemic end"] = kpi_df["Time to epidemic end"]/365
    kpi_df = kpi_df.rename(columns={"Time to epidemic end": "Years to epidemic end"})
    kpi_df.columns = [", ".join(col) if len(col[1])>0 else col[0] 
                              for col in kpi_df.columns]
    percentage_cols = ['Volume, median', 'Volume, upper', 'Volume, lower', 
                   'Symptomatic Max., median', 'Symptomatic Max., upper', 'Symptomatic Max., lower', 
                   'Quarantined Max., median', 'Quarantined Max., upper', 'Quarantined Max., lower',  
                   'Hospital Max., median', 'Hospital Max., upper', 'Hospital Max., lower']
    for col in percentage_cols:
        kpi_df[" (%), ".join(col.split(', '))] = kpi_df[col]/5000
    return (kpi_df.set_index(index_cols_setting)
                  .join(df.groupby(index_cols_setting)[["Run_ID"]]
                          .nunique()
                          .rename(columns={"Run_ID":"# Runs"}))
                  .reset_index())

In [22]:
kpi_df = create_kpi_df(enriched_df)

In [24]:
kpi_df.to_csv("data/public_release/kpi.csv.gz", index=False)
enriched_df.to_csv("data/public_release/data.csv.gz", index=False)